In [ ]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.ml.feature as E

In [ ]:
import os
os.environ['PYSPARK_PYTHON'] = 'C:/Users/DT-Liuxiangfei/.conda/envs/python38/python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:/Users/DT-Liuxiangfei/.conda/envs/python38/python.exe'

In [ ]:
import os
os.environ['HADOOP_HOME'] = 'C:/Users/DT-Liuxiangfei/Documents/TestCodes/model/data/hadoop-3.3.6'
os.environ['hadoop.home.dir'] = 'C:/Users/DT-Liuxiangfei/Documents/TestCodes/model/data/hadoop-3.3.6'

In [ ]:
spark = SparkSession.builder \
    .appName("LocalTest") \
    .master("local[4]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

spark

In [ ]:
# 特征定义保持与原脚本一致
n_host_cols_list = [
    'fre_all_host_10', 'fre_all_host_20', 'fre_all_host_30',
    'fre_ins_host_10', 'fre_ins_host_20', 'fre_ins_host_30',
    'fre_loan_host_10', 'fre_loan_host_20', 'fre_loan_host_30',
    'fre_credit_host_10', 'fre_credit_host_20', 'fre_credit_host_30',
    'fre_car_host_10', 'fre_car_host_20', 'fre_car_host_30',
    'fre_pro_host_10', 'fre_pro_host_20', 'fre_pro_host_30',
    'fre_edu_host_10', 'fre_edu_host_20', 'fre_edu_host_30',
    'fre_inv_host_10', 'fre_inv_host_20', 'fre_inv_host_30',
    'fre_health_host_10', 'fre_health_host_20', 'fre_health_host_30',
    'ins_fre_rate_10', 'ins_fre_rate_20', 'ins_fre_rate_30',
    'loan_fre_rate_10', 'loan_fre_rate_20', 'loan_fre_rate_30',
    'credit_fre_rate_10', 'credit_fre_rate_20', 'credit_fre_rate_30',
    'car_fre_rate_10', 'car_fre_rate_20', 'car_fre_rate_30',
    'pro_fre_rate_10', 'pro_fre_rate_20', 'pro_fre_rate_30',
    'edu_fre_rate_10', 'edu_fre_rate_20', 'edu_fre_rate_30',
    'inv_fre_rate_10', 'inv_fre_rate_20', 'inv_fre_rate_30',
    'health_fre_rate_10', 'health_fre_rate_20', 'health_fre_rate_30',
    'fre_all_host_avg_30', 'fre_all_host_sd_30', 'fre_all_host_cv_30',
    'fre_ins_host_avg_30', 'fre_ins_host_sd_30', 'fre_ins_host_cv_30',
    'fre_loan_host_avg_30', 'fre_loan_host_sd_30', 'fre_loan_host_cv_30',
    'fre_credit_host_avg_30', 'fre_credit_host_sd_30', 'fre_credit_host_cv_30',
    'fre_car_host_avg_30', 'fre_car_host_sd_30', 'fre_car_host_cv_30',
    'fre_pro_host_avg_30', 'fre_pro_host_sd_30', 'fre_pro_host_cv_30',
    'fre_edu_host_avg_30', 'fre_edu_host_sd_30', 'fre_edu_host_cv_30',
    'fre_inv_host_avg_30', 'fre_inv_host_sd_30', 'fre_inv_host_cv_30',
    'fre_health_host_avg_30', 'fre_health_host_sd_30', 'fre_health_host_cv_30',
    'ins_fre_rate_avg_30', 'ins_fre_rate_sd_30', 'ins_fre_rate_cv_30',
    'loan_fre_rate_avg_30', 'loan_fre_rate_sd_30', 'loan_fre_rate_cv_30',
    'credit_fre_rate_avg_30', 'credit_fre_rate_sd_30', 'credit_fre_rate_cv_30',
    'car_fre_rate_avg_30', 'car_fre_rate_sd_30', 'car_fre_rate_cv_30',
    'pro_fre_rate_avg_30', 'pro_fre_rate_sd_30', 'pro_fre_rate_cv_30',
    'edu_fre_rate_avg_30', 'edu_fre_rate_sd_30', 'edu_fre_rate_cv_30',
    'inv_fre_rate_avg_30', 'inv_fre_rate_sd_30', 'inv_fre_rate_cv_30',
    'health_fre_rate_avg_30', 'health_fre_rate_sd_30', 'health_fre_rate_cv_30'
]

numerical_features = ['send_insure_30', 'send_insure_60', 'send_insure_90', 'send_insure_180',
                      'days_since_send_insure_date', 'send_loan_30', 'send_loan_60', 'send_loan_90', 'send_loan_180',
                      'days_since_send_loan_date', 'click_insure_30', 'click_insure_60', 'click_insure_90',
                      'click_insure_180', 'days_since_click_insure_date', 'click_loan_30', 'click_loan_60',
                      'click_loan_90', 'click_loan_180', 'days_since_click_loan_date', 'call_insure_30',
                      'call_insure_60', 'call_insure_90', 'call_insure_180', 'days_since_call_insure_date',
                      'call_loan_30', 'call_loan_60', 'call_loan_90', 'call_loan_180', 'days_since_call_loan_date',
                      'pick_insure_30', 'pick_insure_60', 'pick_insure_90', 'pick_insure_180',
                      'days_since_pick_insure_date', 'pick_loan_30', 'pick_loan_60', 'pick_loan_90', 'pick_loan_180',
                      'days_since_pick_loan_date', 'call_send_insure_30', 'call_send_insure_60', 'call_send_insure_90',
                      'call_send_insure_180', 'days_since_call_send_insure_date', 'call_send_loan_30',
                      'call_send_loan_60', 'call_send_loan_90', 'call_send_loan_180', 'days_since_call_send_loan_date',
                      'call_click_insure_30', 'call_click_insure_60', 'call_click_insure_90', 'call_click_insure_180',
                      'days_since_call_click_insure_date', 'call_click_loan_30', 'call_click_loan_60',
                      'call_click_loan_90', 'call_click_loan_180', 'days_since_call_click_loan_date', ] \
                     + ['city_level', 'new_age', 'active_duration', 'avg_price', 
                        'MAYI_OFFLINE_bx_32140', 'MAYI_OFFLINE_bx_33118', 'MAYI_OFFLINE_bx_32017', 'PROJECT_HS_bx_bx01', 
                        'MAYIYXD_GENERAL_bx_18184', 'MAYIYXD_GENERAL_bx_6134', 'PROJECT_TT_BX_EC_FLOW_RISK_ALIYUN_V21', 'MAYIYXD_GENERAL_bx_18825'] \
                      + n_host_cols_list \
                     + ['duration_30', 'duration_60', 'duration_90', 'duration_120',
                        'interact_30', 'interact_60', 'interact_90', 'interact_120',
                        'say_count_30', 'say_count_60', 'say_count_90', 'say_count_120',
                        'freq_hangup_1_30', 'freq_hangup_1_60', 'freq_hangup_1_90', 'freq_hangup_1_120',
                        'freq_hangup_2_30', 'freq_hangup_2_60', 'freq_hangup_2_90', 'freq_hangup_2_120',
                        'hangup_1_prob_30', 'hangup_1_prob_60', 'hangup_1_prob_90', 'hangup_1_prob_120', ]
categorical_features = ['phone_model', 'browser_family', 'os_family', 'device_brand', 'city_code', 'province_code',
                        'sex', 'hashouse', 'social', 'overdue',
                        'tax', 'married', 'benke', 'kid', 'income', 'consumption', 'shebao']
text_features = ['host_0009', 'host_1019', 'host_2029', 'host_date_0009', 'host_date_1019', 'host_date_2029',
                 'host_fre_0009', 'host_fre_1019', 'host_fre_2029', 'label_0009', 'label_1019', 'label_date_0009',
                 'label_date_1019', 'clicked_products_0009', 'clicked_products_date_0009', 'sms_sent_products_0009',
                 'sms_sent_products_1019', 'sms_sent_products_date_0009', 'sms_sent_products_date_1019',
                 'called_products_0009', 'called_products_1019', 'called_products_date_0009',
                 'called_products_date_1019', 'picked_products_0009', 'picked_products_date_0009',
                 'outbound_sent_products_0009', 'outbound_sent_products_date_0009', ] \
                + ['keypress_30', 'keypress_60', 'keypress_90', 'keypress_120',
                   'rule_name_30', 'rule_name_60', 'rule_name_90', 'rule_name_120',
                   'semantic_30', 'semantic_60', 'semantic_90', 'semantic_120', ] + ['model_value']

In [ ]:
# 数据加载和预处理
df = pd.read_excel("data/train_data.xlsx")
spark_df = spark.createDataFrame(df)
label_data_new = spark_df.drop("hosts", "products", "labels", "keypress", "rules", "semantics")

for col in text_features:
    label_data_new = label_data_new.withColumn(col, F.col(col).cast('string'))

In [ ]:
from pyspark.ml import Pipeline

def save_pipeline_model():
    """
    Save the pipeline model to the specified path.
    """
    string_indexers = [E.StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid='keep') for col in categorical_features]
    one_hot_encoders = [E.OneHotEncoder(inputCols=[col + "_index"], outputCols=[col + "_ohe"], handleInvalid='keep') for col in categorical_features]
    tokenizers = [E.Tokenizer(inputCol=col, outputCol=col + "_token") for col in text_features]
    hashing_tfs = [E.HashingTF(inputCol=col + "_token", outputCol=col + "_hash", numFeatures=16) for col in text_features]
    idfs = [E.IDF(inputCol=col + "_hash", outputCol=col + "_tfidf") for col in text_features]
    median_imputer = E.Imputer(inputCols=['new_age', 'active_duration', 'avg_price'], outputCols=['new_age', 'active_duration', 'avg_price'], strategy="median", missingValue=0)
    
    vector_assembler = E.VectorAssembler(
        inputCols=numerical_features + [col + "_ohe" for col in categorical_features] + [col + "_tfidf" for col in text_features],
        outputCol="features",
        handleInvalid='keep'
    )
    pipeline = Pipeline(stages=string_indexers + one_hot_encoders + tokenizers + hashing_tfs + idfs + [median_imputer, vector_assembler])
    pipeline_model = pipeline.fit(label_data_new)
    return pipeline_model

In [ ]:
pipeline_model = save_pipeline_model()
processed_data = pipeline_model.transform(label_data_new)

In [ ]:
# 保存预处理管道
pipeline_model.write().overwrite().save("model/pipeline_model")

In [ ]:
# 将数据转换为LightGBM可用格式
from scipy.sparse import csr_matrix

# 选择目标列和特征列
df_features = processed_data.select("regged", "features")

# 转换为稀疏矩阵
data, indices, indptr = [], [], [0]
for row in df_features.collect():
    vec = row.features
    data.extend(vec.values)
    indices.extend(vec.indices)
    indptr.append(len(data))

X_sparse = csr_matrix((data, indices, indptr), shape=(df_features.count(), vec.size))
y = np.array(df_features.select("regged").rdd.flatMap(lambda x: x).collect())

In [ ]:
import itertools
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import pickle

# LightGBM参数网格
param_grid = {
    'max_depth': [4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# 计算类别权重
scale_pos_weight = int(pd.Series(y).value_counts()[0] / pd.Series(y).value_counts()[1])

In [ ]:
def grid_search_cv_lgb(param_grid, nfolds):
    best_auc = 0
    best_params = None
    cv_results_all = []
    
    skf = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=42)

    for max_depth, learning_rate, subsample, colsample_bytree in itertools.product(
            param_grid['max_depth'], param_grid['learning_rate'],
            param_grid['subsample'], param_grid['colsample_bytree']):

        params = {
            'objective': 'binary',
            'metric': 'auc',
            'boosting_type': 'gbdt',
            'scale_pos_weight': scale_pos_weight,
            'max_depth': max_depth,
            'learning_rate': learning_rate,
            'subsample': subsample,
            'colsample_bytree': colsample_bytree,
            'random_state': 42,
            'verbose': -1
        }
        
        fold_aucs = []
        
        for train_idx, val_idx in skf.split(X_sparse, y):
            X_train, X_val = X_sparse[train_idx], X_sparse[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            train_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
            
            model = lgb.train(
                params,
                train_data,
                valid_sets=[val_data],
                num_boost_round=1000,
                callbacks=[lgb.early_stopping(20), lgb.log_evaluation(0)]
            )
            
            y_pred = model.predict(X_val, num_iteration=model.best_iteration)
            auc = roc_auc_score(y_val, y_pred)
            fold_aucs.append(auc)
        
        mean_auc = np.mean(fold_aucs)
        cv_results_all.append({
            'params': params,
            'val_auc': mean_auc
        })
        
        print(f"参数: {params}, 平均AUC: {mean_auc:.4f}")
        
        if mean_auc > best_auc:
            best_auc = mean_auc
            best_params = params

    print("最优参数：", best_params)
    print("最优AUC：", best_auc)
    
    return best_params, cv_results_all

In [ ]:
best_params, results = grid_search_cv_lgb(param_grid, 3)

In [ ]:
# 使用最优参数训练最终模型
train_data = lgb.Dataset(X_sparse, label=y)
best_model = lgb.train(
    best_params,
    train_data,
    num_boost_round=1000,
    callbacks=[lgb.log_evaluation(100)]
)

In [ ]:
# 保存模型
with open('model/best_lgb_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

print("模型已保存到 model/best_lgb_model.pkl")

In [ ]:
import matplotlib.pyplot as plt

# 特征重要性分析
feature_importance = best_model.feature_importance(importance_type='gain')
feature_names = [f'feature_{i}' for i in range(len(feature_importance))]

# 获取前20个最重要的特征
top_indices = np.argsort(feature_importance)[-20:]
top_importance = feature_importance[top_indices]
top_names = [feature_names[i] for i in top_indices]

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_importance)), top_importance)
plt.yticks(range(len(top_importance)), top_names)
plt.xlabel('Feature Importance')
plt.title('Top 20 Feature Importance (LightGBM)')
plt.tight_layout()
plt.show()

In [ ]:
# 参数效果分析
results_df = pd.DataFrame(results)
param_names = ['max_depth', 'learning_rate', 'subsample', 'colsample_bytree']

# 分别绘制val_auc关于每个参数的变化关系
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, param in enumerate(param_names):
    grouped = results_df.groupby(results_df['params'].apply(lambda x: x[param]))['val_auc'].mean()
    axes[i].plot(grouped.index, grouped.values, marker='o')
    axes[i].set_xlabel(param)
    axes[i].set_ylabel('val_auc')
    axes[i].set_title(f'val_auc vs {param}')
    axes[i].grid(True)

plt.tight_layout()
plt.show()